# Get the dataset

In [1]:
from typing import Tuple
import numpy as np
import config

dataset_image = np.load(config.x_path, mmap_mode="r")
dataset_label = np.load(config.y_path, mmap_mode="r")
print(dataset_image.shape, dataset_label.shape)
assert(dataset_image.shape[0] == dataset_label.shape[0])
print(f'''Stats:
    | Number of punching: {dataset_label.sum()}
    | Number of not-punching: {dataset_label.shape[0] - dataset_label.sum()}''')


/home/phuonghd/.conda/envs/nhatenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(73058, 3, 224, 224) (73058,)
Stats:
    | Number of not-punching: 38292
    | Number of punching: 34766


# Boilerplate Code

In [2]:
from sklearn.model_selection import RepeatedKFold
import config

kfold = RepeatedKFold(
    n_splits=config.kfold_nsplits,
    n_repeats=config.kfold_nrepeats,
    random_state=config.seed
)


In [3]:
import torch
class Data(torch.utils.data.Dataset):
    def __init__(self, image: np.ndarray, label: np.ndarray, indices: np.ndarray) -> None:
        assert(image.shape[0] == label.shape[0])
        self.image = image
        self.label = label
        self.indices = indices
    
    def __len__(self) -> int:
        return self.indices.shape[0]
    
    def __getitem__(self, idx: int) -> 'Tuple[np.ndarray, bool]':
        idx = self.indices[idx]
        return self.image[idx], self.label[idx]


# Model

In [4]:
from torch import nn


In [5]:
def print_model(model: nn.Module):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    param_size  = param_size / 1024**2
    buffer_size = buffer_size / 1024**2
    print(f'Param size: {param_size:.3f}MB')
    print(f'Buffer size: {buffer_size:.3f}MB')

def print_tensor(tensor: torch.Tensor):
    size_gb = tensor.element_size() * tensor.nelement() / (1<<30)
    print(f"{size_gb:.3f}GB")



In [6]:
model = config.get_model()

use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("CUDA not used!")
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

model = model.to(device)
criterion = criterion.to(device)


# Train

In [7]:
from tqdm import tqdm
def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    train = Data(dataset_image, dataset_label, train_idx)
    train_dataloader = torch.utils.data.DataLoader(
        train, batch_size=config.batch_size)
    total_loss_train = 0
    total_accumulate_train = 0
    for image, label in tqdm(train_dataloader):
        image = image.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.uint8)
        
        output = model(image)
        batch_loss = criterion(output, label)

        total_loss_train += batch_loss

        accumulate = (output.argmax(dim=1) == label).sum()
        total_accumulate_train += accumulate

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

    total_loss_train = total_loss_train.item()
    total_accumulate_train = total_accumulate_train.item()
    return (total_loss_train, total_accumulate_train)


In [8]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    judge = Data(dataset_image, dataset_label, judge_idx)
    judge_dataloader = torch.utils.data.DataLoader(
        judge, batch_size=config.batch_size)

    total_loss_judge = 0
    total_accumulate_judge = 0
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.uint8)

            output = model(image)

            batch_loss = criterion(output, label)
            total_loss_judge += batch_loss

            accumulate = (output.argmax(dim=1) == label).sum()
            total_accumulate_judge += accumulate

    total_loss_judge = total_loss_judge.item()
    total_accumulate_judge = total_accumulate_judge.item()
    return total_loss_judge, total_accumulate_judge

In [9]:
min_judge_loss = float('inf')


for epoch, (train_idx, judge_idx) in enumerate(kfold.split(dataset_label)):
    print(f'''Starting epoch {epoch+1}
        | Train size: {train_idx.shape[0]}
        | Judge size: {judge_idx.shape[0]}''')
    total_loss_train, total_accumulate_train = train(train_idx)
    total_loss_judge, total_accumulate_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
        | Train Loss: {total_loss_train / len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}'''
        
    )
    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        torch.save(model.state_dict(), config.model_path)
        print(f"Save model because val loss improve loss {min_judge_loss:.3f}")

Starting epoch 1
        | Train size: 65752
        | Judge size: 7306


  0%|          | 0/257 [00:00<?, ?it/s]/home/phuonghd/.conda/envs/nhatenv/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py:149: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352630480/work/torch/csrc/utils/tensor_numpy.cpp:172.)
  return default_collate([torch.as_tensor(b) for b in batch])
100%|██████████| 29/29 [00:08<00:00,  3.43it/s]


Epoch: 1 
        | Train Loss: 0.001
        | Train Accuracy: 0.895
        | Val Loss: 0.193
        | Val Accuracy: 0.528
Save model because val loss improve loss 0.193
Starting epoch 2
        | Train size: 65752
        | Judge size: 7306


100%|██████████| 29/29 [00:09<00:00,  3.04it/s]


Epoch: 2 
        | Train Loss: 0.004
        | Train Accuracy: 0.877
        | Val Loss: 0.181
        | Val Accuracy: 0.518
Save model because val loss improve loss 0.181
Starting epoch 3
        | Train size: 65752
        | Judge size: 7306


...

Epoch: 18 
        | Train Loss: 0.003
        | Train Accuracy: 0.470
        | Val Loss: 0.003
        | Val Accuracy: 0.529
Save model because val loss improve loss 0.003
Starting epoch 19
        | Train size: 65753
        | Judge size: 7305


100%|██████████| 29/29 [00:09<00:00,  3.13it/s]


Epoch: 19 
        | Train Loss: 0.003
        | Train Accuracy: 0.468
        | Val Loss: 0.003
        | Val Accuracy: 0.521
Starting epoch 20
        | Train size: 65753
        | Judge size: 7305


100%|██████████| 29/29 [00:09<00:00,  3.19it/s]


Epoch: 20 
        | Train Loss: 0.003
        | Train Accuracy: 0.465
        | Val Loss: 0.003
        | Val Accuracy: 0.530
Save model because val loss improve loss 0.003
